# Video Stabilization
Note: This code is from [SergejVolkov / video_smoothing](https://github.com/SergejVolkov/video_smoothing). All credit for the code itself goes to the authors of this repository. In this notebook, we provide explanatory detail to the  algorithms provided.

In [ ]:
import cv2
import numpy as np
import numpy.linalg as lin

In [ ]:
# Find the best homography that is closest to the current homography model
def get_sub(prev_warp: list, current_warp: np.array, max_n: int) -> tuple:
    best_dist = float("+inf")
    best_warp_ind = -1
    target_warp = lin.inv(current_warp)
    for i in range(len(prev_warp)):
        new_dist = lin.norm(np.dot(prev_warp[max_n // 2], lin.inv(prev_warp[i])) - target_warp)
        if new_dist < best_dist:
            best_dist = new_dist
            best_warp_ind = i
    sub_warp = None
    if best_warp_ind != -1:
        sub_warp = np.dot(prev_warp[max_n // 2], lin.inv(prev_warp[best_warp_ind]))
    return best_warp_ind - max_n // 2, sub_warp

In [ ]:
def smooth_trajectory(cap: cv2.VideoCapture, amount: int) -> tuple:
    max_n = amount
    # List of homographies, starting with the initial frame (identity matrix)
    n_shift = [np.identity(3)]
    apply_warp = []
    prev_frame = None
    curr_ind = 0
    sub_layers = [(0, None)]

    # For each frame
    while True:
        ret, frame_color = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame_color, cv2.COLOR_BGR2GRAY)

        # Extract points to track
        prev_pts = cv2.goodFeaturesToTrack(frame, maxCorners=200,
                                           qualityLevel=0.01, minDistance=30,
                                           blockSize=3)
        if prev_frame is not None:

            # From https://docs.opencv.org/3.4/dc/d6b/group__video__track.html#ga5d10ebbd59fe09c5f650289ec0ece5af
            # "Computes a dense optical flow using the Gunnar Farneback's algorithm."
            # Calculate optical flow (i.e. track feature points)
            curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_frame, frame,
                                                             prev_pts, None)
            
            # Find homography based on matched features using RANSAC algorithm
            M, _ = cv2.findHomography(prev_pts, curr_pts, cv2.RANSAC, 5.0)

            # Local: Homography from previous frame to current frame
            # Global: Homography from first frame to current frame

            # Push the incoming LOCAL homography to list
            n_shift.append(np.dot(M, n_shift[-1]))

            # If there are over max_n homographies, allow a shift for one frame
            # "Shifting" means we disregard the first homography in our list.
            # If we shift multiple times, we will diverge from the first frame's
            # initial camera coordinates.
            if len(n_shift) > max_n:
                n_shift.pop(0)
                inv = lin.inv(n_shift[0])
                for i in range(len(n_shift)):
                    n_shift[i] = np.dot(n_shift[i], inv)
            # We are done with the previous frame, assign the current frame to previous frame
            prev_frame = frame

            # If the current frame is at least the max_n-th frame
            if curr_ind >= max_n:
                # Allocate space by averaging the first half homography models
                apply_warp.append(np.dot(np.average(np.array(n_shift), 0),
                                         lin.inv(np.average(np.array(n_shift[max_n // 2 - 1: max_n // 2 + 1]), 0))))
            elif curr_ind > max_n // 2:
                # Average a few homographies
                apply_warp.append(np.dot(np.average(np.array(n_shift), 0),
                                         lin.inv(np.average(np.array(n_shift[curr_ind - max_n // 2 - 1: curr_ind - max_n // 2 + 1]), 0))))
            elif curr_ind == max_n // 2:
                # Average all homographies
                apply_warp.append(np.average(np.array(n_shift), 0))

            # If we added an averaged homography
            if curr_ind > max_n // 2:
                # Add sub-layer
                sub_layers.append(get_sub(n_shift, apply_warp[-1], max_n))
        else:
            prev_frame = frame
        curr_ind += 1

    # For the first half of frames
    for curr_ind in range(0, max_n // 2):
        # Shift homographies to the left 1
        inv = lin.inv(n_shift.pop(0))
        for i in range(len(n_shift)):
            n_shift[i] = np.dot(n_shift[i], inv)
        # Only average the first half of homographies
        if max_n // 2 < len(n_shift):
            apply_warp.append(np.dot(np.average(np.array(n_shift), 0),
                                     lin.inv(np.average(np.array(n_shift[max_n // 2 - 1: max_n // 2 + 1]), 0))))
        else:
            # else, average all homographies
            apply_warp.append(np.average(np.array(n_shift), 0))
    return apply_warp, sub_layers

In [ ]:
def smooth_cool(cap: cv2.VideoCapture, amount: int) -> tuple:

    # List of homographies, starting with the initial frame (identity matrix)
    n_shift = [np.identity(3)]
    apply_warp = []
    prev_frame = None
    curr_ind = 0
    sub_layers = [(0, None)]

    # For each frame
    while True:
        ret, frame_color = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame_color, cv2.COLOR_BGR2GRAY)

        # Extract points to track
        prev_pts = cv2.goodFeaturesToTrack(frame, maxCorners=200,
                                           qualityLevel=0.01, minDistance=30,
                                           blockSize=3)
        if prev_frame is not None:

            # From https://docs.opencv.org/3.4/dc/d6b/group__video__track.html#ga5d10ebbd59fe09c5f650289ec0ece5af
            # "Computes a dense optical flow using the Gunnar Farneback's algorithm."
            # Calculate optical flow (i.e. track feature points)
            curr_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_frame, frame,
                                                             prev_pts, None)
            
            # Find homography based on matched features using RANSAC algorithm
            M, _ = cv2.findHomography(prev_pts, curr_pts, cv2.RANSAC, 5.0)

            # Local: Homography from previous frame to current frame
            # Global: Homography from first frame to current frame

            # Push the incoming LOCAL homography to list
            n_shift.append(np.dot(M, n_shift[-1]))

            # If there are over "amount" homographies, allow a shift for one frame
            # "Shifting" means we disregard the first homography in our list.
            # If we shift multiple times, we will diverge from the first frame's
            # initial camera coordinates.
            if len(n_shift) > amount:
                n_shift.pop(0)
                inv = lin.inv(n_shift[0])
                for i in range(len(n_shift)):
                    n_shift[i] = np.dot(n_shift[i], inv)

            # We are dome with the previous frame, assign the current frame to previous frame
            prev_frame = frame

            # If the current frame is at least the "amount"th frame
            if curr_ind >= amount:
                # Allocate space by averaging the first half homography models
                apply_warp.append(np.dot(np.average(np.array(n_shift), 0),
                                         lin.inv(n_shift[amount // 2])))
            # Else if the current frame is at least the "amount / 2"th frame
            elif curr_ind >= amount // 2:

                # Average a few homographies
                apply_warp.append(np.dot(np.average(np.array(n_shift), 0),
                                         lin.inv(n_shift[curr_ind - amount // 2])))
            # If we added an averaged homography
            if curr_ind > amount // 2:
                # Add sub-layer
                sub_layers.append(get_sub(n_shift, apply_warp[-1], amount))
        else:
            prev_frame = frame
        curr_ind += 1

    # For the first half of frames
    for curr_ind in range(0, amount // 2):
        # Shift homographies to the left 1
        n_shift.pop(0)
        inv = lin.inv(n_shift[0])
        for i in range(len(n_shift)):
            n_shift[i] = np.dot(n_shift[i], inv)
        if amount // 2 < len(n_shift):
            # Only average the first half of homographies
            apply_warp.append(np.dot(np.average(np.array(n_shift), 0),
                                     lin.inv(n_shift[amount // 2])))
        else:
            # Average all
            apply_warp.append(np.average(np.array(n_shift), 0))
    return apply_warp, sub_layers

In [ ]:
'''
    cap: Video capture to be processed
    amount: Distance threshold to skip stablization and shift current and future frames
'''
def stabilize(cap: cv2.VideoCapture, amount: int) -> None:

    # amount - I don't know what this does
    amount = amount - amount % 2

    print("Processing data...")

    # Get homographies and sub layers
    # apply_warp, sub_layers = smooth_cool(cap, amount)
    apply_warp, sub_layers = smooth_trajectory(cap, amount)

    print("Applying smoothing...")

    cap.set(2, 0)
    frames = []

    # Read all frames
    while True:
        ret, frame_color = cap.read()
        if not ret:
            break
        frames.append(frame_color)

    # Define the codec for output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Get frames per second
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Width and height of input video
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Initialize output video
    out = cv2.VideoWriter('output.mp4', fourcc, fps, (w, h))

    print("Writing to disk...")

    # For each frame
    for curr_ind in range(len(frames)):

        # Warp current frame with respective homography model
        frame_color = cv2.warpPerspective(frames[curr_ind], apply_warp[curr_ind], (w, h))

        # If a sub-layer does not exist for the current frame
        if curr_ind < len(sub_layers) and sub_layers[curr_ind][1] is not None:
            mask = cv2.warpPerspective(np.full(frame_color.shape, 255, dtype=np.uint8),
                                       apply_warp[curr_ind], (w, h)) != 255
            np.putmask(frame_color, mask,
                       cv2.warpPerspective(frames[curr_ind + sub_layers[curr_ind][0]],
                                           np.dot(apply_warp[curr_ind],
                                                  sub_layers[curr_ind][1]), (w, h)))
        out.write(frame_color)
        #cv2.imshow('frame', frame_color)
        #if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break

    out.release()
    print("Done")

In [ ]:
import cv2

cap = cv2.VideoCapture("input.mp4")
# cap = cv2.VideoCapture("test2.MOD")
# cap = cv2.VideoCapture("test3.MOD")
# cap = cv2.VideoCapture("test4.MOD")
# cap = cv2.VideoCapture("test5.MOV")

stabilize(cap, 1000)

cap.release()
cv2.destroyAllWindows()

Processing data...
Applying smoothing...
Writing to disk...
Done
